# Gradioでデモ構築

In [ ]:
# !pip install datasets evaluate transformers[sentencepiece]
# !pip install gradio

## Gradioの構文に慣れよう

1. メインの関数を定義する：ここではgreet()という関数。機械学習のアプリケーションでは、この関数がモデルを呼び出して入力に対して予測を行い、その出力を返す。
2. fn、inputs、outputsの3つの引数を持つGradio Interfaceを作成する：これらの引数は、入力と出力のコンポーネントのタイプを定義します。下の例では、両方のコンポーネントが単純なテキストボックスになっています。
3. 作成した Interface に対して launch() メソッドを呼び出す


In [ ]:
import gradio as gr


def greet(name):
    return "Hello " + name


demo = gr.Interface(fn=greet, inputs="text", outputs="text")

demo.launch()

入力コンポーネントとしてクラスオブジェクトをインスタンス化することができます。
ここではTextboxという入力コンポーネントで、ラベル、プレースホルダー、行数を設定した入力テキストボックスを作成できます。

出力コンポーネントも同様にいじれます。

In [ ]:
import gradio as gr


def greet(name):
    return "Hello " + name


# We instantiate the Textbox class
textbox = gr.Textbox(label="Type your name here:", placeholder="John Doe", lines=2)

gr.Interface(fn=greet, inputs=textbox, outputs="text").launch()

GPT-2のようなテキスト生成モデルをデモするための簡単なインターフェースを作ってみましょう。

まず、テキストプロンプトを取り込み、テキスト補完を返す予測関数を定義します。

In [ ]:

# GPT-2を使ったテキスト生成を実装してみてください。
def predict(prompt):
    completion = None
    return completion


gr.Interface(fn=predict, inputs="text", outputs="text").launch()

# Interface class詳細

Interfaceの作成に必須である引数
1. fn: Gradioインターフェースによってラップされる関数。
2. inputs: 入力コンポーネントのタイプ（複数可）。Gradioは、"image "や "mic "などの多くのビルド済みコンポーネントを提供してくれます。
3. outputs: 出力コンポーネントの種類（複数可）。入力と同様。

### オーディオを使った簡単な例
地味にすごいのはマイクとスピーカーを制御して音声を処理できることです。ここでは、オーディオファイルを受け取り、それを単純に反転させるaudio-to-audio関数を作成します。

入力には、Audio コンポーネントを使用します。Audioコンポーネントを使用する場合、オーディオのソースとして、ユーザがアップロードするファイルか、ユーザが自分の声を録音するマイクかを指定することができます。今回は、「マイク」に設定してみましょう。

さらに、音声をnumpyの配列として受け取り、簡単に「逆引き」できるようにしたいと思います。そこで、"type "を "numpy "に設定し、入力データを(sample_rate, data)のタプルとして関数に渡します。

また、サンプルレートとnumpy配列のデータを持つタプルを、再生可能なオーディオファイルとして自動的にレンダリングできるAudio出力コンポーネントを使用します。この場合、カスタマイズは必要ないので、文字列のショートカットである "audio "を使用します。

In [ ]:
import numpy as np
import gradio as gr

def reverse_audio(audio):
    sr, data = audio  # 音声データは大体このように扱ってきた。それが埋め込まれているのは便利。
    reversed_audio = (sr, np.flipud(data))  # https://note.nkmk.me/python-numpy-flip-flipud-fliplr/
    return reversed_audio


mic = gr.Audio(source="microphone", type="numpy", label="Speak here...")
gr.Interface(reverse_audio, mic, "audio").launch()

### 複数の入出力の処理
もっと複雑な、複数の入力と出力を持つ関数があるとしましょう。以下の例では、ドロップダウンのインデックス、スライダーの値、および数値を受け取り、音楽の音声サンプルを返す関数があります。

入出力コンポーネントのリストをどのように渡しているかを見て、何が起こっているかを追ってみてください。

ここで重要なのは、入力コンポーネントのリストを渡すとき

1. 入力コンポーネントのリスト、各コンポーネントは順番にパラメータに対応する。
2. 出力コンポーネントのリストでは、各コンポーネントが返される値に対応します。
3. 以下のコードでは、3つの入力コンポーネントがgenerate_tone()関数の3つの引数にどのように並んでいるのかを示しています。

In [ ]:
import numpy as np
import gradio as gr

notes = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"]  # 音の名前、indexがノートナンバー


def generate_tone(note, octave, duration):
    sr = 48000  # sr: sampling frequency, サンプリング周波数。１秒間に48000点のデータ点をもつ。
    a4_freq = 440  # A4音、ラの音（https://acoustics.jp/qanda/answer/168.html）
    tones_from_a4 = 12 * (octave - 4) + (note - 9)  # オクターブは12音なので、鍵盤のどこのブロックか。notesのA（９）を基準に、ブロック内のどこか。https://web.quizknock.com/octave
    frequency = a4_freq * 2 ** (tones_from_a4 / 12) # 生成したい音の周波数を計算（https://www.asahi-net.or.jp/~hb9t-ktd/music/Japan/Research/DTM/freq_map.html）
    duration = int(duration)
    audio = np.linspace(0, duration, duration * sr)  # https://note.nkmk.me/python-numpy-arange-linspace/
    audio = (20000 * np.sin(audio * (2 * np.pi * frequency))).astype(np.int16)  # http://www.slp.k.hosei.ac.jp/~itou/lecture/2011/DigitalData/01_text.pdf
    return (sr, audio)


gr.Interface(
    generate_tone,
    [
        gr.Dropdown(notes, type="index"),
        gr.Slider(minimum=4, maximum=6, step=1),
        gr.Number(type="number", value=1, label="Duration in seconds"),
    ],
    "audio",
).launch()

音声認識は、このような処理を土台に前処理を行います（大変だった）。Transformer以降、音声認識の分野ではE2Eモデルと言われていましたが、前処理も簡素になりました。以前は、言語モデルだけでなく音素辞書や発音辞書などが必要だったり、Deeplearningのモデルは一部での利用でした。全てを取り払ってDeeplearningで解決してしまうのが当たり前になりましたが、チューニングのしやすさなどから、以前の音声認識技術はまだまだ活用されそうです。

launch()の引数を試してみてください。
- inline=False
- inbrowser=True
- share=True

share=Trueで、公開された共有可能なリンクが生成され、誰にでも送ることができます。このリンクを送ると、相手側のユーザーは最大72時間、自分のブラウザでモデルを試用することができます。処理はあなたのデバイスで行われるため（デバイスの電源が入っている限り！）、依存関係のパッケージングを心配する必要はありません。Google Colabノートブックで作業している場合、共有リンクは常に自動的に作成されます。通常はこのような形です。XXXXX.gradio.app. このリンクはGradioのリンクを通じて提供されますが、私たちはあなたのローカルサーバーのプロキシに過ぎず、インターフェースを通じて送られたデータを保存することはありません。

In [ ]:
from transformers import pipeline
import gradio as gr

model = pipeline("automatic-speech-recognition")


def transcribe_audio(mic=None, file=None):
    if mic is not None:
        audio = mic
    elif file is not None:
        audio = file
    else:
        return "You must either provide a mic recording or a file"
    transcription = model(audio)["text"]
    return transcription


gr.Interface(
    fn=transcribe_audio,
    inputs=[
        gr.Audio(source="microphone", type="filepath", optional=True),
        gr.Audio(source="upload", type="filepath", optional=True),
    ],
    outputs="text",
).launch()

ここで入力した`音声データはダウンロード`しておいてください。後で使います。

# デモをシェアする

デモだけあっても不親切なので、案内情報を使いできるようになっています。

![fig](https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter9/gradio-demo-overview.png)

- title: 入出力コンポーネントの上に表示されるデモのタイトル
- description: インターフェイスの説明 (テキスト、Markdown または HTML) 
- article: インターフェイスを説明する記事(テキスト、Markdown、HTMLのいずれか)。入力と出力のコンポーネントの下に表示されます。
- theme: テーマ（default, huggingface, grass, peach）。- dark- prefix を追加することもできます。
- examples: あなたのデモをより使いやすくするために、関数への入力例をいくつか用意することができます。これらはネストしたリストとして提供する必要があり、外側のリストはサンプルで構成され、内側のリストは各入力コンポーネントに対応する入力で構成されています。
- live: デモを「ライブ」にしたい場合、つまり、入力が変わるたびにモデルを再実行したい場合は、live=True を設定します。

In [ ]:
title = "Ask Rick a Question"
description = """
The bot was trained to answer questions based on Rick and Morty dialogues. Ask Rick anything!
<img src="https://huggingface.co/spaces/course-demos/Rick_and_Morty_QA/resolve/main/rick.png" width=200px>
"""

article = "Check out [the original Rick and Morty Bot](https://huggingface.co/spaces/kingabzpro/Rick_and_Morty_Bot) that this demo is based off of."

gr.Interface(
    fn=predict,
    inputs="textbox",
    outputs="text",
    title=title,
    description=description,
    article=article,
    examples=[["What are you doing?"], ["Where should we time travel to?"]],
).launch()

## Hugging Face Spacesでデモをホストする

共有リンクは一時的なもので実際には使い物にはなりません。永続的なホスティング環境が必要です。

Hugging Face Spacesは、Gradioデモをインターネット上で恒久的にホストするためのインフラを、無料で提供します。Spacesは、あなたのGradioインターフェイスのコードがapp.pyファイルに存在する（パブリックまたはプライベート）レポを作成し、そこにプッシュすることを可能にします。

以下はスケッチ認識のデモで、HuggingFaceを使わない例です。以下のファイルをダウンロードする必要があります。

- [class_names.txt](https://huggingface.co/spaces/dawood/Sketch-Recognition/blob/main/class_names.txt)
- [pytorch_model.bin](https://huggingface.co/spaces/dawood/Sketch-Recognition/blob/main/pytorch_model.bin)

In [ ]:
# !wget https://huggingface.co/spaces/course-demos/Sketch-Recognition/raw/main/class_names.txt
# !wget https://huggingface.co/spaces/course-demos/Sketch-Recognition/resolve/main/pytorch_model.bin

In [ ]:
from pathlib import Path
import torch
import gradio as gr
from torch import nn

LABELS = Path("class_names.txt").read_text().splitlines()

model = nn.Sequential(
    nn.Conv2d(1, 32, 3, padding="same"),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(32, 64, 3, padding="same"),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(64, 128, 3, padding="same"),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(1152, 256),
    nn.ReLU(),
    nn.Linear(256, len(LABELS)),
)
state_dict = torch.load("pytorch_model.bin", map_location="cpu")
model.load_state_dict(state_dict, strict=False)
model.eval()


def predict(im):
    x = torch.tensor(im, dtype=torch.float32).unsqueeze(0).unsqueeze(0) / 255.0
    with torch.no_grad():
        out = model(x)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    values, indices = torch.topk(probabilities, 5)
    return {LABELS[i]: v.item() for i, v in zip(indices, values)}

このinputs="sketchpad"はサブミットボタンなしにデータをモデルに行って、推論を出すことができます。

In [ ]:
interface = gr.Interface(
    predict,
    inputs="sketchpad",
    outputs="label",
    theme="huggingface",
    title="Sketch Recognition",
    description="Who wants to play Pictionary? Draw a common object like a shovel or a laptop, and the algorithm will guess in real time!",
    article="<p style='text-align: center'>Sketch Recognition | Demo Model</p>",
    live=True,
)
interface.launch(share=True)

# Hugging Face Hubとの連携

Interface.load() メソッドを使用して、"model/" (もしくは "huggingface/") とモデル名を続けて渡します。

これによって、モデルを直接ロードすることができます。piplineと同等の処理が実装されています。

それだけでなく、メモリにモデルをロードしていません。Hugging Faceの推論APIを使用しています。これはGPT-JやT0ppのような、多くのRAMを必要とする巨大なモデルに理想的です。


In [ ]:
import gradio as gr

title = "GPT-J-6B"
description = "Gradio Demo for GPT-J 6B, a transformer model trained using Ben Wang's Mesh Transformer JAX. 'GPT-J' refers to the class of model, while '6B' represents the number of trainable parameters. To use it, simply add your text, or click one of the examples to load them. Read more at the links below."
article = "<p style='text-align: center'><a href='https://github.com/kingoflolz/mesh-transformer-jax' target='_blank'>GPT-J-6B: A 6 Billion Parameter Autoregressive Language Model</a></p>"
examples = [
    ["The tower is 324 metres (1,063 ft) tall,"],
    ["The Moon's orbit around Earth has"],
    ["The smooth Borealis basin in the Northern Hemisphere covers 40%"],
]
gr.Interface.load(
    "huggingface/EleutherAI/gpt-j-6B",
    inputs=gr.Textbox(lines=5, label="Input Text"),
    title=title,
    description=description,
    article=article,
    examples=examples,
    enable_queue=True,
).launch()

Hugging Faceハブから任意のSpaceをロードしてローカルに再作成することも一発でできます。

 `Interface` に `spaces/` を渡し、その後にSpaceの名前を指定することができます。

In [ ]:
gr.Interface.load("spaces/abidlabs/remove-bg").launch()


HubやSpacesからデモを読み込む際に便利なのが、パラメータをオーバーライドしてカスタマイズすることです。ここでは、タイトルを追加し、代わりにWebカメラで動作するようにしています。

In [14]:
gr.Interface.load(
    "spaces/abidlabs/remove-bg", inputs="webcam", title="Remove your webcam background!"
).launch()

Fetching Space from: https://huggingface.co/spaces/abidlabs/remove-bg


/Users/hajime/projects/ai_lab/.venv/lib/python3.9/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Users/hajime/projects/ai_lab/.venv/lib/python3.9/site-packages/gradio/interface.py:329: UserWarning: Currently, only the 'default' theme is supported.
  warnings.warn("Currently, only the 'default' theme is supported.")


Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


# Advanced Interface features

ここまでで基本的な機能を動かしてきました。ここではstateを導入します。

## state: データを持続させるためにステートを使用する
Gradioは、ページロード内の複数のサブミットにわたってデータを永続化するセッションステートをサポートしています。セッションステートは、例えばチャットボットのデモを作成する際に、ユーザーがモデルとインタラクトする際にデータを持続させたい場合に便利です。

セッションステートにデータを保存するには、3つのことを行う必要があります。

1. 関数に、インターフェースの状態を表す追加パラメータを渡す。
2. 関数の最後で、状態の更新された値を追加の戻り値として返す。
3. インターフェースの作成時に、「state」入力コンポーネントと「state」出力コンポーネントを追加します。


In [ ]:
import random

import gradio as gr


def chat(message, history):
    history = history or []
    if message.startswith("How many"):
        response = random.randint(1, 10)
    elif message.startswith("How"):
        response = random.choice(["Great", "Good", "Okay", "Bad"])
    elif message.startswith("Where"):
        response = random.choice(["Here", "There", "Somewhere"])
    else:
        response = "I don't know"
    history.append((message, response))
    return history, history


iface = gr.Interface(
    chat,
    ["text", "state"],
    ["chatbot", "state"],
    allow_screenshot=False,
    allow_flagging="never",
)
iface.launch()

# Blocks class
Interface classとは違う、低レベルAPIであるBlocks classはデータフローとレイアウトを制御できます。

- Interface：入力と出力のリストを提供するだけで、完全な機械学習デモを作成できる高レベルのAPIです。
- Blocks：Blocks（「積み木」の意）を使って、非常に複雑で多段階のアプリケーションを構築することができます。


以下のような要求に応えることができます。
- 関連するデモを1つのWebアプリケーションで複数のタブとしてグループ化したい。
- デモのレイアウトを変更したい（例：入出力の位置を指定したい）。
- あるモデルの出力が次のモデルの入力になるようなマルチステップインターフェースなど、より柔軟なデータフローが欲しい。
- ユーザーの入力に基づき、コンポーネントのプロパティ（例えば、ドロップダウンの選択肢）やその可視性を変更したい。

In [ ]:
import gradio as gr


def flip_text(x):
    return x[::-1]


demo = gr.Blocks()

with demo:
    gr.Markdown(
        """
    # Flip Text!
    Start typing below to see the output.
    """
    )
    input = gr.Textbox(placeholder="Flip this text")
    output = gr.Textbox()

    input.change(fn=flip_text, inputs=input, outputs=output)

demo.launch()

この簡単な例では、Blocksの根底にある4つの概念を紹介しています。

1. Blocksを使うと、with gradio.Blocksコンテキスト内でPythonのオブジェクトをインスタンス化するだけで、マークダウン、HTML、ボタン、インタラクティブなコンポーネントを組み合わせたWebアプリケーションを構築することができるようになります。コンポーネントをインスタンス化する順番は、各要素が作成された順番にWebアプリにレンダリングされるため、重要です。(より複雑なレイアウトについては後述します)

2. 通常のPython関数は、コードのどこにでも定義でき、ブロックを使ってユーザーの入力で実行することができます。この例では入力されたテキストを「反転」させる簡単な関数を書いていますが、単純な計算から機械学習モデルによる予測処理まで、あらゆるPython関数を書くことができます。

3. Blocksのコンポーネントには、イベントを割り当てることができます。これにより、そのコンポーネントがクリックされたり、変更されたりしたときに、関数が実行されます。イベントを割り当てる際には、fn: 呼び出される関数、inputs: 入力コンポーネントのリスト、outputs: 呼び出される出力コンポーネントのリストという3つのパラメータを渡すことになります。

4. Blocksは、定義したイベントトリガーをもとに、コンポーネントがインタラクティブ（ユーザーからの入力を受け付ける）であるべきかどうかを自動的に判断します。この例では、最初のテキストボックスの値は flip_text() 関数で使用されるため、インタラクティブになります。2 番目のテキストボックスは、その値が入力として使用されることはないので、インタラクティブではありません。これを上書きしたい場合は、コンポーネントの interactive パラメータにブール値を渡します（例： gr.Textbox(placeholder="Flip this text", interactive=True))。

## デモのレイアウトをカスタマイズする
デフォルトでは、Blocksは作成したコンポーネントを1列に縦に並べてレンダリングします。これを変更するには、 gradio.Column(): で列を、 gradio.Row(): で行を追加し、それらのコンテキスト内でコンポーネントを作成します。

ここで注意しなければならないのは、Column の下に作成されたコンポーネント（これはデフォルトでもあります）は、縦にレイアウトされるということです。Rowの下に作成されたコンポーネントは、ウェブ開発におけるフレックスボックスモデルと同様に、水平方向にレイアウトされます。

最後に、with gradio.Tabs()コンテキストマネージャを使用して、デモ用のタブを作成することもできます。このコンテキスト内では、with gradio.TabItem(name_of_tab): children を指定することで複数のタブを作成することができます。with gradio.TabItem(name_of_tab): コンテキスト内に作成されたコンポーネントは、そのタブに表示されます。

In [ ]:
import numpy as np
import gradio as gr

demo = gr.Blocks()


def flip_text(x):
    return x[::-1]


def flip_image(x):
    return np.fliplr(x)


with demo:
    gr.Markdown("Flip text or image files using this demo.")
    with gr.Tabs():
        with gr.TabItem("Flip Text"):
            with gr.Row():
                text_input = gr.Textbox()
                text_output = gr.Textbox()
            text_button = gr.Button("Flip")
        with gr.TabItem("Flip Image"):
            with gr.Row():
                image_input = gr.Image()
                image_output = gr.Image()
            image_button = gr.Button("Flip")

    text_button.click(flip_text, inputs=text_input, outputs=text_output)
    image_button.click(flip_image, inputs=image_input, outputs=image_output)

demo.launch()

## イベントと状態を調べる
レイアウトをコントロールするのと同じように、Blocksではどのイベントが関数呼び出しのトリガーになるかを細かくコントロールすることができます。各コンポーネントや多くのレイアウトには、それぞれ対応する特定のイベントがあります。

例えばTextboxコンポーネントには、change() （テキストボックス内の値が変化したとき）とsubmit() （テキストボックスにフォーカスした状態でユーザがエンターキーを押したとき）の2つのイベントがあります。より複雑なコンポーネントでは、さらに多くのイベントを持つことができます。たとえば、Audioコンポーネントでは、オーディオファイルが再生、クリア、一時停止されたときなどにも別のイベントが発生します。各コンポーネントがサポートするイベントについては、ドキュメントを参照してください。

イベントトリガーは、これらのイベントのいずれか、または複数にアタッチすることができます。イベントトリガーを作成するには、コンポーネントインスタンスのイベント名を関数として呼び出します（例： textbox.change(...) や btn.click(...) ）。この関数は、前述したように3つのパラメータを受け取ります。

- fn: 実行する関数
- inputs: 関数の入力パラメータとして値を与えるコンポーネント（のリスト）。各コンポーネントの値は、順番に対応する関数パラメータにマップされます。関数がパラメータを取らない場合、このパラメータは None にすることができます。
- outputs: 関数から返される値に基づいて値が更新されるコンポーネント（のリスト）。各戻り値は、順番に、対応するコンポーネントの値を設定します。このパラメータは、関数が何も返さない場合は None にすることができます。

GPTモデルを使ってテキスト補完を行うこの例のように、入力と出力のコンポーネントを同じコンポーネントにすることもできます。

In [ ]:
import gradio as gr

api = gr.Interface.load("huggingface/EleutherAI/gpt-j-6B")


def complete_with_gpt(text):
    # Use the last 50 characters of the text as context
    return text[:-50] + api(text[-50:])


with gr.Blocks() as demo:
    textbox = gr.Textbox(placeholder="Type here and press enter...", lines=4)
    btn = gr.Button("Generate")

    # イベントトリガー
    btn.click(complete_with_gpt, textbox, textbox)

demo.launch()

## マルチステップ・デモの作成
ある関数の出力を次の関数の入力として再利用するような、マルチステップのデモを実装します。あるイベント・トリガーの入力にコンポーネントを使い、別のイベント・トリガーの出力にコンポーネントを使うことができるので、これはBlockでとても簡単に実現できます。

以下の例のtextコンポーネントを見てください。その値は音声テキストモデルの結果ですが、センチメント分析モデルにも渡されます。

In [ ]:
from transformers import pipeline

import gradio as gr

asr = pipeline("automatic-speech-recognition", "facebook/wav2vec2-base-960h")
classifier = pipeline("text-classification")


def speech_to_text(speech):
    text = asr(speech)["text"]
    return text


def text_to_sentiment(text):
    return classifier(text)[0]["label"]


demo = gr.Blocks()

with demo:
    audio_file = gr.Audio(type="filepath")
    text = gr.Textbox()
    label = gr.Label()

    b1 = gr.Button("Recognize Speech")
    b2 = gr.Button("Classify Sentiment")

    b1.click(speech_to_text, inputs=audio_file, outputs=text)
    b2.click(text_to_sentiment, inputs=text, outputs=label)

demo.launch()

## コンポーネントのプロパティを更新する
他のコンポーネントの値を更新するためのイベントを作成する方法について見てきました。

しかし、テキストボックスの可視性やラジオボタングループの選択肢のように、コンポーネントの他のプロパティを変更したい場合はどうすればよいのでしょうか。この場合、関数から通常の戻り値ではなく、コンポーネントクラスの update() メソッドを返すことで対応できます。


In [ ]:
import gradio as gr


def change_textbox(choice):
    if choice == "short":
        return gr.Textbox.update(lines=2, visible=True)
    elif choice == "long":
        return gr.Textbox.update(lines=8, visible=True)
    else:
        return gr.Textbox.update(visible=False)


with gr.Blocks() as block:
    radio = gr.Radio(
        ["short", "long", "none"], label="What kind of essay would you like to write?"
    )
    text = gr.Textbox(lines=2, interactive=True)

    radio.change(fn=change_textbox, inputs=radio, outputs=text)
    block.launch()